<a href="https://colab.research.google.com/github/JamesBolt22/Supervised_Contrastive_learning_for_onset_detection/blob/main/Shulter_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import h5py
import keras.backend as K
import glob

In [ ]:
#@title Plot Spec
def plot_spec(spectrogram):
    
    fig, ax = plt.subplots(nrows=1, sharex=True, figsize=(10,10))
    cax = ax.matshow(spectrogram, aspect='auto', origin='lower')

In [ ]:
#@title Load fold data

with h5py.File('/content/drive/MyDrive/data_fold_0/train_data.h5', 'r') as hf:
    train_data = hf["input_data"][:]
with h5py.File('/content/drive/MyDrive/data_fold_0/train_labels_0.020.h5', 'r') as hf:
    train_label = hf["input_data"][:]

with h5py.File('/content/drive/MyDrive/data_fold_0/val_data.h5', 'r') as hf:
    val_data = hf["input_data"][:]
with h5py.File('/content/drive/MyDrive/data_fold_0/val_label.h5', 'r') as hf:
    val_label = hf["input_data"][:]

with h5py.File('/content/drive/MyDrive/data_fold_0/weights_data_0.020.h5', 'r') as hf:
    train_weights = hf["input_data"][:]    

In [ ]:
#@title Combine val data
val_full = (val_data, val_label)

In [ ]:
#@title Create full model
def new_model(input_shape, projection_size):  

  inputs = keras.Input(shape=input_shape)

  x = tf.keras.layers.Conv2D(filters=10, kernel_size=(3,7), strides=1, activation="relu")(inputs)

  x = tf.keras.layers.MaxPooling2D(pool_size=(3,1))(x)

  x = tf.keras.layers.Conv2D(filters=20, kernel_size=(3,3), strides=1, activation="relu")(x)

  x = tf.keras.layers.MaxPooling2D(pool_size=(3,1))(x)

  x = tf.keras.layers.Flatten()(x)

  x = tf.keras.layers.Dense(256, activation="relu")(x)
  x  = tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer = tf.keras.optimizers.Adam() ,loss = tf.keras.losses.BinaryCrossentropy(),metrics=tf.keras.metrics.BinaryAccuracy())

  return model

In [ ]:
#@title Set Params
hop_len = 441
nfft = 256
num_of_bins = 80
size_of_slice = 14
input_shape = (num_of_bins,size_of_slice+1,3)
batch_size = 256
epoch_num = 150

In [ ]:
#@title Create model
model = new_model(input_shape, projection_size)
model.summary()

In [ ]:
#@title Fit model
history = model.fit(x = train_data,  y = train_label, batch_size = batch_size, epochs=epoch_num, validation_data = val_full, shuffle=True,sample_weight=train_weights)

In [ ]:
#@title Save model
model.save("/content/drive/MyDrive/Masters/Models/innacurate/bock_0.12")